In [1]:
import pyspark
import numpy as np
import pandas as pd
import os
from os import walk
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import array_contains
from pyspark.sql.functions import *
import time
from pyspark.sql.functions import date_format
from datetime import date
import datetime
from datetime import datetime
import re

date = date(2023,2,27)                  ## to be changed as per requirement
day = date.strftime('%d')
nummonth=date.strftime("%m")
year=date.strftime('%Y')
tablename="r"+str(day)+str(nummonth)+str(year)
print(tablename)

st = time.time()

def finnifty_data():
    spark = SparkSession.builder.config("spark.jars", "C:\\Users\\admin\\Downloads\\postgresql-42.5.0.jar") \
    .master("local").appName("PySpark_Postgres_test").getOrCreate()
    #spark.sparkContext.setLogLevel("WARN")
    df = spark.read.format("jdbc").option("url", "jdbc:postgresql://swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com/RawDataBase").option("user","postgres").option("password","swancap123")\
        .option("driver", "org.postgresql.Driver").option("dbtable", tablename)\
        .option("user", "postgres").option("password", "swancap123").load()
    ## GETTING ONLY TIME IN TIME COLUMN
    q = df.withColumn('time',date_format('time', 'HH:mm:ss'))
    ## FILTERING FINNIFTY DATA
    fndata = q.filter(q.ticker.contains('FINNIFTY') & ((q.ticker.endswith('E.NFO'))| (q.ticker.endswith('E'))))
    ## REPLACING .NFO IN ticker
    fndata = fndata.withColumn('ticker',regexp_replace('ticker','.NFO',''))
    fndata = fndata.drop(col('ticker_check'))
    ## CONVERTING PYSPARK DATAFRAME TO PANDAS DATAFRAME
    fndata=fndata.toPandas()
    return fndata

def finnifty_monthly():
    exp_df = pd.read_excel(r"C:\Users\admin\Desktop\Expiry_Dates\Finnifty.xlsx",sheet_name='Monthly')
    exp_df.rename({'curr_date': 'New_date'}, axis=1, inplace=True)
    exp_date = pd.read_excel(r"C:\Users\admin\Desktop\Expiry_Dates\Finnifty_Monthly_Expiry.xlsx")

    folpath = r"C:\Users\admin\Desktop\Pyspark_Contracts\FinNifty\Monthly_Data\\"
    finalpath = r"C:\Users\admin\Desktop\Pyspark\FinNifty\Monthly\\"
    s = 'FINNIFTY'
    def add(stri):
        obj = datetime.strptime(stri, "%b")
        month_number = obj.month
        return month_number
    sym = 'FINNIFTY'
    fndata = finnifty_data()
    dff = fndata.copy()
    dff = dff[dff['ticker'].str.contains('FINNIFTY')]
    dff = dff[(dff['ticker'].str.endswith('E')) | (dff['ticker'].str.endswith('E.NFO'))]
    dff = dff.loc[:, ~dff.columns.str.contains('^Unnamed')]
    dff['time'] = pd.to_datetime(dff['time']).dt.time
    dff['date'] = pd.to_datetime(dff['date'],dayfirst=True)
    dff['ticker'] = dff['ticker'].str.replace('30MAR23','29MAR23',regex=True)

    dff['Option_Type'] = dff['ticker'].str[-2:]
    dff['Temp'] = dff["ticker"].str.replace(s,"")
    dff['Temp'] = dff['Temp'].str[:-2]
    dff['Length_of_Temp'] = dff['Temp'].str.len()
    dff['Strike'] = np.where((dff['Temp'].str.len()==9) | (dff['Temp'].str.len()==11) , 
                              dff['Temp'].str[-4:] , 
                              dff['Temp'].str[-5:])
    dff['Exp_Year'] = np.where((dff['Temp'].str.len()==9) | (dff['Temp'].str.len()==10) ,
                               dff['Temp'].str[:2] ,
                               dff['Temp'].str[5:7])
    dff['Exp_month'] = dff['Temp'].str[2:5]

    dff['Exp_Year'] = dff['Exp_Year'].astype('str')
    dff['MonthYear'] = dff['Exp_month']+dff['Exp_Year']
    dff = pd.merge(dff,exp_date,on='MonthYear')
    dff = dff.drop(['MonthYear','Month','Year'],axis=1)
    dff = dff.rename(columns={'Exp_DT':'Monthly_Expiry'})

    dff['Length_of_Temp'] = dff['Length_of_Temp'].astype('int64')
    temp_10 = dff[(dff['Length_of_Temp']==10) | (dff['Length_of_Temp']==9)]

    temp_12 = dff[(dff['Length_of_Temp']==12) | (dff['Length_of_Temp']==11)]
    temp_12['DateDate'] = temp_12['Temp'].str[:2]
    temp_12['DateDate'] = temp_12['DateDate'].astype('int64')
    temp_12['Exp_DT'] = pd.to_datetime(temp_12['Monthly_Expiry'],dayfirst=True)
    temp_12['Exp_Day'] = temp_12['Exp_DT'].dt.day
    temp_12 = temp_12[temp_12['Exp_Day']==temp_12['DateDate']]
    temp_12 = temp_12.drop(['DateDate','Exp_Day'],axis=1)

    temp_df = temp_10.append(temp_12,ignore_index=True)
    temp_df['exp_month_number'] = temp_df.apply(lambda row : add(row["Exp_month"]), axis = 1)
    temp_df['New_date'] = temp_df['date']
    temp_df["New_date"] = pd.to_datetime(temp_df["New_date"])
    
    temp_df["current_month_number"] = temp_df['New_date'].dt.month
    temp_df["difference"] = temp_df['exp_month_number'].astype(int) - temp_df["current_month_number"].astype(int)

    temp_df1 = pd.merge(temp_df, 
                     exp_df, 
                     on ='New_date', 
                     how ='left')
    temp_df1.drop(temp_df1.filter(regex="Unname"),axis=1, inplace=True)
    temp_df1["current_exp_month_number"] = temp_df1['curr_exp_date'].dt.month
    temp_df1["Diff_months"] = temp_df1["current_exp_month_number"] - temp_df1["current_month_number"]
    temp_df1["Diff_months"] = temp_df1["Diff_months"].astype(int) 
    temp_df1['Current_Year'] = temp_df1['New_date'].dt.year.astype(str).str[-2:]

    bdf = temp_df1[temp_df1["Diff_months"] == 0]
    adf = temp_df1[(temp_df1["Diff_months"] == 1) | (temp_df1["Diff_months"] == -11)]
    agb = adf.groupby(["difference"])
    unique_val_list_a = list(adf["difference"].unique())

    bgb = bdf.groupby(["difference"])
    unique_val_list_b = list(bdf["difference"].unique())

    ## TO AVOID OVERWRITING, REMOVING FILE
    for i in range(3):
        if(i==0):
            file='-I'
        if(i==1):
            file='-II'
        if(i==2):
            file='-III'
        if(os.path.exists(folpath+sym+file+".csv")):
            os.remove(folpath+sym+file+".csv")

    for i in unique_val_list_b:
        temp_df_new = bgb.get_group(i)
        temp_df_new = temp_df_new.drop(temp_df_new.columns[9:],axis=1)
        if i == 0:
            temp_df_new.to_csv(folpath + sym + '-I.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-I.csv'), index=False)

        if i == 1 or i == -11:
            temp_df_new.to_csv(folpath + sym + '-II.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-II.csv'), index=False)

        if i == 2 or i == -10:
            temp_df_new.to_csv(folpath + sym + '-III.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-III.csv'), index=False)

    for i in unique_val_list_a:
        temp_df_new = agb.get_group(i)
        temp_df_new = temp_df_new.drop(temp_df_new.columns[9:],axis=1)
        if i == 1 or i == -11:
            temp_df_new.to_csv(folpath + sym + '-I.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-I.csv'), index=False)

        if i == 2 or i == -10:
            temp_df_new.to_csv(folpath + sym + '-II.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-II.csv'), index=False)

        if i == 3 or i == -9:
            temp_df_new.to_csv(folpath + sym + '-III.csv', mode = 'a', header = not os.path.exists(folpath + sym + '-III.csv'), index=False)

    for i in range(3):
        if(i==0):
            file='-I'
        if(i==1):
            file='-II'
        if(i==2):
            file='-III'

        if os.path.exists(finalpath+sym+file+".csv"):
            os.remove(finalpath+sym+file+".csv")
        if os.path.exists(folpath+sym+file+".csv"):
            df = pd.read_csv(folpath+sym+file+".csv")
            df['Option_Type'] = df['ticker'].str[-2:]
            df['Strike'] = np.where((df['ticker'].str.len()==19) | (df['ticker'].str.len()==21) , df['ticker'].str[-6:-2] , df['ticker'].str[-7:-2])
            df['Symbol'] = 'FINNIFTYMONTHLY' + file + df['Strike'].astype(int).astype(str) + df['Option_Type']
            df['ticker'] = df['Symbol']
            df = df.drop(df.columns[9:],axis=1)
            df = df.sort_values(by='date')
            print(file,df.shape[0])
            df = df.drop_duplicates()
            print(file,df.shape[0])
            df.to_csv(finalpath+sym+file+".csv",index=False)
    print("FINNIFTY MONTHLY CONTRACTS GENERATED")
    
def finnifty_weekly():
    exp_df1 = pd.read_excel(r"C:\Users\admin\Desktop\Expiry_Dates\Finnifty.xlsx",sheet_name='Weekly',parse_dates=['date'],usecols= ['date','Week_number'])
    exp_df2 = pd.read_excel(r"C:\Users\admin\Desktop\Expiry_Dates\Finnifty.xlsx",sheet_name='Weekly',parse_dates=['Weekly_Expiry_Date'],usecols= ['Weekly_Expiry_Date', 'Expiry_Week_number'])
    exp_date = pd.read_excel(r"C:\Users\admin\Desktop\Expiry_Dates\Finnifty_Monthly_Expiry.xlsx")

    folpath = r"C:\Users\admin\Desktop\Pyspark_Contracts\FinNifty\Weekly_Data\\"
    finalpath = r"C:\Users\admin\Desktop\Pyspark\FinNifty\Weekly\\"
    s = 'FINNIFTY'
    def add(stri):
        obj = datetime.strptime(stri, "%b")
        month_number = obj.month
        return month_number
    sym = 'FINNIFTY'
    fndata = finnifty_data()
    dff = fndata.copy()
    dff = dff[dff['ticker'].str.contains('FINNIFTY')]
    dff = dff[(dff['ticker'].str.endswith('E')) | (dff['ticker'].str.endswith('E.NFO'))]
    dff = dff.loc[:, ~dff.columns.str.contains('^Unnamed')]
    dff['time'] = pd.to_datetime(dff['time']).dt.time
    dff['date'] = pd.to_datetime(dff['date'],dayfirst=True)
    dff['ticker'] = dff['ticker'].str.replace('30MAR23','29MAR23',regex=True)

    dff['Option_Type'] = dff['ticker'].str[-2:]
    dff['Temp'] = dff["ticker"].str.replace(s,"")
    dff['Temp'] = dff['Temp'].str[:-2]
    dff['Length_of_Temp'] = dff['Temp'].str.len()
    dff['Strike'] = np.where((dff['Temp'].str.len()==9) | (dff['Temp'].str.len()==11) , 
                              dff['Temp'].str[-4:] , 
                              dff['Temp'].str[-5:])
    dff['Exp_Year'] = np.where((dff['Temp'].str.len()==9) | (dff['Temp'].str.len()==10) ,
                               dff['Temp'].str[:2] ,
                               dff['Temp'].str[5:7])
    dff['Exp_month'] = dff['Temp'].str[2:5]
    dff['EXPIRY_DT'] = dff['Temp'].str[:7]
    dff['EXPIRY_DT'] = pd.to_datetime(dff['EXPIRY_DT'],dayfirst=True)

    dff['Exp_Year'] = dff['Exp_Year'].astype('str')
    dff['MonthYear'] = dff['Exp_month']+dff['Exp_Year']
    dff = pd.merge(dff,exp_date,on='MonthYear')
    dff = dff.drop(['MonthYear','Month','Year'],axis=1)
    dff = dff.rename(columns={'Exp_DT':'Monthly_Expiry'})

    ## GETTING EXPIRY DATES FOR MONTHLY CONTRACTS
    dff['expiry_date'] = np.where(dff['Length_of_Temp']>=11,dff['EXPIRY_DT'],dff['Monthly_Expiry'])
    dff = dff.drop(['EXPIRY_DT',"Monthly_Expiry"],axis=1)
    ## GETTING WEEK NUMBER OF CURRENT DATE
    dff = dff.rename(columns={'Date':'date'})
    dff = pd.merge(dff,exp_df1,on='date',how='left')

    ## GETTING WEEK NUMBER OF EXPIRY DATES
    exp_df2['Weekly_Expiry_Date'] = pd.to_datetime(exp_df2['Weekly_Expiry_Date'],dayfirst=True)
    exp_df2 = exp_df2.dropna()
    exp_df2 = exp_df2.rename(columns = {'Weekly_Expiry_Date':'expiry_date'})
    temp_df = pd.merge(dff, exp_df2, on = 'expiry_date', how = 'left')
    temp_df = temp_df.drop_duplicates()
    temp_df['week_diff'] = temp_df['Expiry_Week_number'] - temp_df['Week_number']
    final_df = temp_df.copy()
    final_df["week_diff"] = final_df['week_diff'].replace(np.nan,10000)
    
    agb = final_df.groupby(["week_diff"])
    unique_val_list_a = list(final_df["week_diff"].unique())
    unique_val_list_a = sorted([a for a in unique_val_list_a if a>=0])[0:14]
    print(unique_val_list_a)
    for i in range(12):
        if(i==0):
            file='-I'
        if(i==1):
            file='-II'
        if(i==2):
            file='-III'
        if(i==3):
            file='-IV'
        if(i==4):
            file='-V'
        if(i==5):
            file='-VI'
        if(i==6):
            file='-VII'
        if(i==7):
            file='-VIII'
        if(i==8):
            file='-IX'
        if(i==9):
            file='-X'
        if(i==10):
            file='-XI'
        if(i==11):
            file='-XII'
            
        if(os.path.exists(folpath+sym+file+".csv")):
            os.remove(folpath+sym+file+".csv")

    for i in sorted(unique_val_list_a):
        temp_df = agb.get_group(i)
        temp_df = temp_df.drop(temp_df.columns[9:],axis=1)
        if i == 0:
            temp_df.to_csv(folpath + s + '-I.csv', mode = 'a', header = not os.path.exists(folpath + s + '-I.csv'), index=False)

        if i == 1:
            temp_df.to_csv(folpath + s + '-II.csv', mode = 'a', header = not os.path.exists(folpath + s + '-II.csv'), index=False)

        if i == 2:
            temp_df.to_csv(folpath + s + '-III.csv', mode = 'a', header = not os.path.exists(folpath + s + '-III.csv'), index=False)

        if i == 3:
            temp_df.to_csv(folpath + s + '-IV.csv', mode = 'a', header = not os.path.exists(folpath + s + '-IV.csv'), index=False)

        if i == 4:
            temp_df.to_csv(folpath + s + '-V.csv', mode = 'a', header = not os.path.exists(folpath + s + '-V.csv'), index=False)

        if i == 5:
            temp_df.to_csv(folpath + s + '-VI.csv', mode = 'a', header = not os.path.exists(folpath + s + '-VI.csv'), index=False)

        if i == 6:
            temp_df.to_csv(folpath + s + '-VII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-VII.csv'), index=False)

        if i == 7:
            temp_df.to_csv(folpath + s + '-VIII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-VIII.csv'), index=False)

        if i == 8:
            temp_df.to_csv(folpath + s + '-IX.csv', mode = 'a', header = not os.path.exists(folpath + s + '-IX.csv'), index=False)

        if i == 9:
            temp_df.to_csv(folpath + s + '-X.csv', mode = 'a', header = not os.path.exists(folpath + s + '-X.csv'), index=False)

        if i == 10:
            temp_df.to_csv(folpath + s + '-XI.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XI.csv'), index=False)

        if i == 11:
            temp_df.to_csv(folpath + s + '-XII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XII.csv'), index=False)

        if i == 12:
            temp_df.to_csv(folpath + s + '-XIII.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XIII.csv'), index=False)

        if i == 13:
            temp_df.to_csv(folpath + s + '-XIV.csv', mode = 'a', header = not os.path.exists(folpath + s + '-XIV.csv'), index=False)

    for i in range(12):
        if(i==0):
            file='-I'
        if(i==1):
            file='-II'
        if(i==2):
            file='-III'
        if(i==3):
            file='-IV'
        if(i==4):
            file='-V'
        if(i==5):
            file='-VI'
        if(i==6):
            file='-VII'
        if(i==7):
            file='-VIII'
        if(i==8):
            file='-IX'
        if(i==9):
            file='-X'
        if(i==10):
            file='-XI'
        if(i==11):
            file='-XII'
        if(i==12):
            file='-XIII'
        if(i==13):
            file='-XIV'
        if os.path.exists(finalpath+sym+file+".csv"):
            os.remove(finalpath+sym+file+".csv")
        if(os.path.exists(folpath+sym+file+".csv")):
            print(file)
            df = pd.read_csv(folpath+sym+str(file)+".csv")
            df['Option_Type'] = df['ticker'].str[-2:]
            df['Strike'] = np.where((df['ticker'].str.len()==19) | (df['ticker'].str.len()==21) , df['ticker'].str[-6:-2] , df['ticker'].str[-7:-2])
            df['Symbol'] = 'FINNIFTY' + file + df['Strike'].astype(int).astype(str) + df['Option_Type']
            df['ticker'] = df['Symbol']
            df = df.drop(df.columns[9:],axis=1)
            df = df.sort_values(by='date')
            print(df.shape[0])
            df = df.drop_duplicates()
            print(df.shape[0])
            df.to_csv(finalpath+sym+str(file)+".csv",index=False)
        else:
            print(file,"not exists")
    print("FINNIFTY WEEKLY CONTRACTS CREATED")
    
print("FINNIFTY CONTRACTS BEING CREATED")
finnifty_monthly()
finnifty_weekly()
et = time.time()

print("Elapsed Time",et-st)

r27022023
FINNIFTY CONTRACTS BEING CREATED


C:\Users\admin\AppData\Local\Temp\ipykernel_7404\586304269.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_10.append(temp_12,ignore_index=True)


-I 22910
-I 22910
-II 7
-II 7
-III 1
-III 1
FINNIFTY MONTHLY CONTRACTS GENERATED
[0.0, 1.0, 2.0, 3.0, 4.0, 10000.0]
-I
22910
22910
-II
5057
5057
-III
21
21
-IV
3
3
-V
7
7
-VI not exists
-VII not exists
-VIII not exists
-IX not exists
-X not exists
-XI not exists
-XII not exists
FINNIFTY WEEKLY CONTRACTS CREATED
Elapsed Time 28.550374031066895
